In [1]:
org = "420D598021E0084A07C98EC91DCAE0B880287912A925799429825980593D7DCD400820329480BF21003CC0086028910097520230C80813401D8CC00F601881805705003CC00E200E98400F50031801D160048E5AFEFD5E5C02B93F2F4C11CADBBB799CB294C5FDB8E12C40139B7C98AFA8B2600DCBAF4D3A4C27CB54EA6F5390B1004B93E2F40097CA2ECF70C1001F296EF9A647F5BFC48C012C0090E675DF644A675DF645A7E6FE600BE004872B1B4AAB5273ED601D2CD240145F802F2CFD31EFBD4D64DD802738333992F9FFE69CAF088C010E0040A5CC65CD25774830A80372F9D78FA4F56CB6CDDC148034E9B8D2F189FD002AF3918AECD23100953600900021D1863142400043214C668CB31F073005A6E467600BCB1F4B1D2805930092F99C69C6292409CE6C4A4F530F100365E8CC600ACCDB75F8A50025F2361C9D248EF25B662014870035600042A1DC77890200D41086B0FE4E918D82CC015C00DCC0010F8FF112358002150DE194529E9F7B9EE064C015B005C401B8470F60C080371460CC469BA7091802F39BE6252858720AC2098B596D40208A53CBF3594092FF7B41B3004A5DB25C864A37EF82C401C9BCFE94B7EBE2D961892E0C1006A32C4160094CDF53E1E4CDF53E1D8005FD3B8B7642D3B4EB9C4D819194C0159F1ED00526B38ACF6D73915F3005EC0179C359E129EFDEFEEF1950005988E001C9C799ABCE39588BB2DA86EB9ACA22840191C8DFBE1DC005EE55167EFF89510010B322925A7F85A40194680252885238D7374C457A6830C012965AE00D4C40188B306E3580021319239C2298C4ED288A1802B1AF001A298FD53E63F54B7004A68B25A94BEBAAA00276980330CE0942620042E3944289A600DC388351BDC00C9DCDCFC8050E00043E2AC788EE200EC2088919C0010A82F0922710040F289B28E524632AE0"

In [34]:
def decode_packet(txt, packets, idx ):
    ver = int(txt[idx: idx + 3], 2)
    typ = int(txt[idx + 3: idx + 6], 2)
    idx += 6
    if (typ == 4):
        data = []
        cont = True
        while txt[idx] == '1':
            data.append(txt[idx + 1: idx + 5])
            idx += 5

        data.append(txt[idx + 1: idx + 5])
        idx += 5

        packets.append({'typ': typ,'ver':ver, 'value': int("".join(data),2)})
        return idx         
            
    else:
        len_id = txt[idx]
        idx += 1
        pc = []
        if(len_id == '0'):
            total_len = int(txt[idx: idx + 15], 2)
            idx += 15
            target = idx + total_len
            while idx < target:
                idx = decode_packet(txt , pc, idx )
                
            packets.append({'typ':typ, 'ver':ver, 'sub': pc.copy()})
        else:
            num_sub = int(txt[idx: idx + 11], 2)
            idx += 11
            for j in range(num_sub):
                idx = decode_packet(txt, pc, idx)

            packets.append({'typ':typ, 'ver':ver, 'sub': pc.copy()})

        return idx 
        

In [35]:
s = org
txt = "".join([f"{int(si, 16):04b}" for si in s])
p = []
decode_packet(txt,p,0)

5380

In [89]:
def count_versions(p):
    versions = 0
    def dfs(node):
        nonlocal versions
        if not node:
            return
        versions += node['ver']
        if 'sub' in node:
            for n in node['sub']:
                dfs(n)
    dfs(p[0])
    return versions
        

In [90]:
print(count_versions(p))

991


In [91]:
import functools

def calculate(p):
    def dfs(node):
        if not node:
            return
        tp = node['typ']
        if tp == 4:
            return node['value']
        if 'sub' in node:
            values= [dfs(n) for n in node['sub']]
            if tp == 0:
                return sum(values)
            elif tp == 1:
                return functools.reduce(lambda a, b: a * b, values)
            elif tp == 2:
                return min(values)
            elif tp == 3:
                return max(values)
            elif tp == 5:
                return 1 if values[0] > values[1] else 0
            elif tp == 6:
                return 1 if values[0] < values[1] else 0
            elif tp == 7:
                return 1 if values[0] == values[1] else 0
    return dfs(p[0])

In [92]:
test = [
("C200B40A82" , 3),
("04005AC33890" , 54),
("880086C3E88112" , 7),
("CE00C43D881120" , 9),
("D8005AC2A8F0" , 1),
("F600BC2D8F", 0),
("9C005AC2F8F0" , 0),
("9C0141080250320F1802104A08", 1)
]

In [93]:
for s, target in test:
    txt = "".join([f"{int(si, 16):04b}" for si in s])
    p = []
    decode_packet(txt,p,0)
    assert calculate(p) == target

In [94]:
s = org
txt = "".join([f"{int(si, 16):04b}" for si in s])
p = []
decode_packet(txt,p , 0)

5380

In [70]:
count_versions(p)

991


In [71]:
calculate(p)

1264485568252